# Setup Elastic Client

In [1]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

es = Elasticsearch(["http://localhost:9200"], http_auth=('elastic', 'xxxxxx'), timeout=30)
es.cluster.health(wait_for_status='yellow', request_timeout=10)

/home/toor/semantic-search/.venv/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'cluster_name': 'docker-cluster',
 'status': 'green',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 4,
 'active_shards': 4,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 0,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 100.0}

# Load data

In [2]:
!pip install pysbd

In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

In [25]:
# file = "Трудовой кодекс+8.02.2023.xlsx"
file = "fixed 10.02.23.xlsx"
df = pd.read_excel(file, engine="openpyxl")
df.head()

,Unnamed: 0,party,section_number,section_text,chapter_number,chapter_text,article_number,article_text,text,href
0,501,ЧАСТЬ ПЯТАЯ,Раздел XIII,ЗАЩИТА ТРУДОВЫХ ПРАВ И СВОБОД.\nРАССМОТРЕНИЕ И...,Глава 61,РАССМОТРЕНИЕ И РАЗРЕШЕНИЕ КОЛЛЕКТИВНЫХ\nТРУДОВ...,Статья 405,Гарантии в связи с разрешением коллективного т...,"Члены примирительной комиссии, трудовые арбитр...",http://www.consultant.ru/document/cons_doc_LAW...
1,137,ЧАСТЬ ТРЕТЬЯ,Раздел V,ВРЕМЯ ОТДЫХА,Глава 19,ОТПУСКА,Статья 126,Замена ежегодного оплачиваемого отпуска денежн...,"Часть ежегодного оплачиваемого отпуска, превыш...",http://www.consultant.ru/document/cons_doc_LAW...
2,405,ЧАСТЬ ЧЕТВЕРТАЯ,Раздел XII,ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА\nОТДЕЛЬНЫХ КАТ...,Глава 53.1,"ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА РАБОТНИКОВ,\nН...",Статья 341.1,Общие положения,Частное агентство занятости или другое юридиче...,http://www.consultant.ru/document/cons_doc_LAW...
3,0,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 1,Цели и задачи трудового законодательства,Целями трудового законодательства являются уст...,http://www.consultant.ru/document/cons_doc_LAW...
4,253,ЧАСТЬ ТРЕТЬЯ,Раздел X,ОХРАНА ТРУДА,Глава 36,УПРАВЛЕНИЕ ОХРАНОЙ ТРУДА\n(введено Федеральным...,Статья 225,Финансирование мероприятий по улучшению услови...,Финансирование мероприятий по улучшению услови...,http://www.consultant.ru/document/cons_doc_LAW...


## Some cleanup

In [26]:
df.text = df.text.apply(lambda x: x.replace("_x000D_\n", " "))
df.section_text = df.section_text.apply(lambda x: x.replace("_x000D_\n", " "))
df.chapter_text = df.chapter_text.apply(lambda x: x.replace("_x000D_\n", " "))
df.article_text = df.article_text.apply(lambda x: x.replace("_x000D_\n", " "))

df.section_text = df.section_text.apply(lambda x: x.replace("\n", " "))
df.chapter_text = df.chapter_text.apply(lambda x: x.replace("\n", " "))
df.article_text = df.article_text.apply(lambda x: x.replace("\n", " "))

In [27]:
df.head()

,Unnamed: 0,party,section_number,section_text,chapter_number,chapter_text,article_number,article_text,text,href
0,501,ЧАСТЬ ПЯТАЯ,Раздел XIII,ЗАЩИТА ТРУДОВЫХ ПРАВ И СВОБОД. РАССМОТРЕНИЕ И ...,Глава 61,РАССМОТРЕНИЕ И РАЗРЕШЕНИЕ КОЛЛЕКТИВНЫХ ТРУДОВЫ...,Статья 405,Гарантии в связи с разрешением коллективного т...,"Члены примирительной комиссии, трудовые арбитр...",http://www.consultant.ru/document/cons_doc_LAW...
1,137,ЧАСТЬ ТРЕТЬЯ,Раздел V,ВРЕМЯ ОТДЫХА,Глава 19,ОТПУСКА,Статья 126,Замена ежегодного оплачиваемого отпуска денежн...,"Часть ежегодного оплачиваемого отпуска, превыш...",http://www.consultant.ru/document/cons_doc_LAW...
2,405,ЧАСТЬ ЧЕТВЕРТАЯ,Раздел XII,ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА ОТДЕЛЬНЫХ КАТЕ...,Глава 53.1,"ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА РАБОТНИКОВ, НА...",Статья 341.1,Общие положения,Частное агентство занятости или другое юридиче...,http://www.consultant.ru/document/cons_doc_LAW...
3,0,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 1,Цели и задачи трудового законодательства,Целями трудового законодательства являются уст...,http://www.consultant.ru/document/cons_doc_LAW...
4,253,ЧАСТЬ ТРЕТЬЯ,Раздел X,ОХРАНА ТРУДА,Глава 36,УПРАВЛЕНИЕ ОХРАНОЙ ТРУДА (введено Федеральным ...,Статья 225,Финансирование мероприятий по улучшению услови...,Финансирование мероприятий по улучшению услови...,http://www.consultant.ru/document/cons_doc_LAW...


In [28]:
s = """
Если иное не установлено настоящим Кодексом, другими федеральными законами, при заключении трудового договора лицо, поступающее на работу, предъявляет работодателю:
(в ред. Федерального закона от 01.12.2014 N 409-ФЗ)
паспорт или иной документ, удостоверяющий личность;
трудовую книжку и (или) сведения о трудовой деятельности (статья 66.1 настоящего Кодекса), за исключением случаев, если трудовой договор заключается впервые;
(в ред. Федерального закона от 16.12.2019 N 439-ФЗ)
документ, подтверждающий регистрацию в системе индивидуального (персонифицированного) учета, в том числе в форме электронного документа;
(в ред. Федерального закона от 01.04.2019 N 48-ФЗ)
документы воинского учета - для военнообязанных и лиц, подлежащих призыву на военную службу;
документ об образовании и (или) о квалификации или наличии специальных знаний - при поступлении на работу, требующую специальных знаний или специальной подготовки;
(в ред. Федерального закона от 02.07.2013 N 185-ФЗ)
справку о наличии (отсутствии) судимости и (или) факта уголовного преследования либо о прекращении уголовного преследования по реабилитирующим основаниям, выданную в порядке и по форме, которые устанавливаются федеральным органом исполнительной власти, осуществляющим функции по выработке и реализации государственной политики и нормативно-правовому регулированию в сфере внутренних дел, - при поступлении на работу, связанную с деятельностью, к осуществлению которой в соответствии с настоящим Кодексом, иным федеральным законом не допускаются лица, имеющие или имевшие судимость, подвергающиеся или подвергавшиеся уголовному преследованию;
(абзац введен Федеральным законом от 23.12.2010 N 387-ФЗ)
справку о том, является или не является лицо подвергнутым административному наказанию за потребление наркотических средств или психотропных веществ без назначения врача либо новых потенциально опасных психоактивных веществ, которая выдана в порядке и по форме, которые устанавливаются федеральным органом исполнительной власти, осуществляющим функции по выработке и реализации государственной политики и нормативно-правовому регулированию в сфере внутренних дел, - при поступлении на работу, связанную с деятельностью, к осуществлению которой в соответствии с федеральными законами не допускаются лица, подвергнутые административному наказанию за потребление наркотических средств или психотропных веществ без назначения врача либо новых потенциально опасных психоактивных веществ, до окончания срока, в течение которого лицо считается подвергнутым административному наказанию.
(абзац введен Федеральным законом от 13.07.2015 N 230-ФЗ)
В отдельных случаях с учетом специфики работы настоящим Кодексом, иными федеральными законами, указами Президента Российской Федерации и постановлениями Правительства Российской Федерации может предусматриваться необходимость предъявления при заключении трудового договора дополнительных документов.
Запрещается требовать от лица, поступающего на работу, документы помимо предусмотренных настоящим Кодексом, иными федеральными законами, указами Президента Российской Федерации и постановлениями Правительства Российской Федерации.
КонсультантПлюс: примечание.
С 01.01.2023 в ч. 4 ст. 65 вносятся изменения (ФЗ от 14.07.2022 N 240-ФЗ). См. будущую редакцию.
При заключении трудового договора впервые работодателем оформляется трудовая книжка (за исключением случаев, если в соответствии с настоящим Кодексом, иным федеральным законом трудовая книжка на работника не оформляется). В случае, если на лицо, поступающее на работу впервые, не был открыт индивидуальный лицевой счет, работодателем представляются в соответствующий территориальный орган Пенсионного фонда Российской Федерации сведения, необходимые для регистрации указанного лица в системе индивидуального (персонифицированного) учета.
(в ред. Федеральных законов от 01.04.2019 N 48-ФЗ, от 16.12.2019 N 439-ФЗ)
В случае отсутствия у лица, поступающего на работу, трудовой книжки в связи с ее утратой, повреждением или по иной причине работодатель обязан по письменному заявлению этого лица (с указанием причины отсутствия трудовой книжки) оформить новую трудовую книжку (за исключением случаев, если в соответствии с настоящим Кодексом, иным федеральным законом трудовая книжка на работника не ведется).
(часть пятая введена Федеральным законом от 30.06.2006 N 90-ФЗ; в ред. Федерального закона от 16.12.2019 N 439-ФЗ)
КонсультантПлюс: примечание.
О ведении трудовых книжек в бумажном или электронном виде после 31.12.2020 см. ст. 2 ФЗ от 16.12.2019 N 439-ФЗ.



Трудовые отношения возникают между работником и работодателем на основании трудового договора, заключаемого ими в соответствии с настоящим Кодексом.
В случаях и порядке, которые установлены трудовым законодательством и иными нормативными правовыми актами, содержащими нормы трудового права, или уставом (положением) организации, трудовые отношения возникают на основании трудового договора в результате:
(в ред. Федерального закона от 30.06.2006 N 90-ФЗ)
избрания на должность;
(в ред. Федерального закона от 30.06.2006 N 90-ФЗ)
избрания по конкурсу на замещение соответствующей должности;
назначения на должность или утверждения в должности;
направления на работу уполномоченными в соответствии с федеральным законом органами в счет установленной квоты;
(в ред. Федерального закона от 30.06.2006 N 90-ФЗ)
судебного решения о заключении трудового договора;
абзац утратил силу. - Федеральный закон от 30.06.2006 N 90-ФЗ;
признания отношений, связанных с использованием личного труда и возникших на основании гражданско-правового договора, трудовыми отношениями.
(абзац введен Федеральным законом от 28.12.2013 N 421-ФЗ)
Трудовые отношения между работником и работодателем возникают также на основании фактического допущения работника к работе с ведома или по поручению работодателя или его уполномоченного на это представителя в случае, когда трудовой договор не был надлежащим образом оформлен.
(часть третья введена Федеральным законом от 30.06.2006 N 90-ФЗ, в ред. Федерального закона от 28.12.2013 N 421-ФЗ)
Фактическое допущение работника к работе без ведома или поручения работодателя либо его уполномоченного на это представителя запрещается.
(часть четвертая введена Федеральным законом от 28.12.2013 N 421-ФЗ)

время отпусков по уходу за ребенком до достижения им установленного законом возраста;
абзац утратил силу. - Федеральный закон от 22.07.2008 N 157-ФЗ.
В стаж работы, дающий право на ежегодные дополнительные оплачиваемые отпуска за работу с вредными и (или) опасными условиями труда, включается только фактически отработанное в соответствующих условиях время.

Сверхурочная работа оплачивается за первые два часа работы не менее чем в полуторном размере, за последующие часы - не менее чем в двойном размере. Конкретные размеры оплаты за сверхурочную работу могут определяться коллективным договором, локальным нормативным актом или трудовым договором. По желанию работника сверхурочная работа вместо повышенной оплаты может компенсироваться предоставлением дополнительного времени отдыха, но не менее времени, отработанного сверхурочно, за исключением случаев, предусмотренных настоящим Кодексом.
Часть вторая утратила силу. - Федеральный закон от 30.06.2006 N 90-ФЗ.
Работа, произведенная сверх нормы рабочего времени в выходные и нерабочие праздничные дни и оплаченная в повышенном размере либо компенсированная предоставлением другого дня отдыха в соответствии со статьей 153 настоящего Кодекса, не учитывается при определении продолжительности сверхурочной работы, подлежащей оплате в повышенном размере в соответствии с частью первой настоящей статьи.

Правительством Российской Федерации федеральным органом исполнительной власти.
Часть седьмая утратила силу с 1 января 2015 года. - Федеральный закон от 22.12.2014 N 443-ФЗ.
При избрании работника по конкурсу на замещение ранее занимаемой
"""
note_cleanup_re: re.Pattern = re.compile(r'КонсультантПлюс\: примечание\.\n.*?\n', flags=re.MULTILINE)
ref_cleanup_re: re.Pattern = re.compile(r'\(.*?N\s+\d+-ФЗ.*?\)\n')
removed_cleanup_re: re.Pattern = re.compile(r'абзац утратил силу\..*?(;|\.)\n')
removed2_cleanup_re: re.Pattern = re.compile(r'Часть \w+ утратила силу(\sс.*?года)?. - Федеральный закон от .*?\n')
def general_cleanup(s):
    s = note_cleanup_re.sub('', s)
    s = ref_cleanup_re.sub('', s)
    s = removed_cleanup_re.sub('', s)
    s = removed2_cleanup_re.sub('', s)
    return s


print(general_cleanup(s))



Если иное не установлено настоящим Кодексом, другими федеральными законами, при заключении трудового договора лицо, поступающее на работу, предъявляет работодателю:
паспорт или иной документ, удостоверяющий личность;
трудовую книжку и (или) сведения о трудовой деятельности (статья 66.1 настоящего Кодекса), за исключением случаев, если трудовой договор заключается впервые;
документ, подтверждающий регистрацию в системе индивидуального (персонифицированного) учета, в том числе в форме электронного документа;
документы воинского учета - для военнообязанных и лиц, подлежащих призыву на военную службу;
документ об образовании и (или) о квалификации или наличии специальных знаний - при поступлении на работу, требующую специальных знаний или специальной подготовки;
справку о наличии (отсутствии) судимости и (или) факта уголовного преследования либо о прекращении уголовного преследования по реабилитирующим основаниям, выданную в порядке и по форме, которые устанавливаются федеральным органом 

In [29]:
df.text = df.text.apply(lambda x: general_cleanup(x))

In [30]:
df[df.text.str.contains('утратила силу')]

,Unnamed: 0,party,section_number,section_text,chapter_number,chapter_text,article_number,article_text,text,href


In [31]:
# def remove_link(row):
#     for r in [x for x in re.findall(r"\((.*?)\)", row) if re.search(r"\d\d\.\d\d\.\d\d\d\d", x)]:
#         row = row.replace(f"({r})", "")
#     return row

import pysbd

def get_sentence_spans(text):
    seg = pysbd.Segmenter(language="ru", clean=True, char_span=False)
    r = seg.segment(text)
    r.reverse()
    bullets_fixed_r = []
    
    def reccurent_bullet_list(prefix=''):
        while len(r)>0:
            x = r.pop()
            if x.endswith(';'):
                bullets_fixed_r.append(prefix+' '+x)
            elif x.endswith(':'):
                reccurent_bullet_list(prefix+' '+x[:-1])
                break
            else:
                bullets_fixed_r.append(prefix+' '+x)
                break
        

    while len(r)>0:
        s = r.pop()
        if s.endswith(':'):
            reccurent_bullet_list(s[:-1])
        else:
            bullets_fixed_r.append(s)
                
    return bullets_fixed_r
        

print(df.text[73])
# get_sentence_spans(remove_link(df.text[338]))
get_sentence_spans(general_cleanup(df.text[73]))

С руководителями государственных и муниципальных научных организаций заключаются трудовые договоры на срок до пяти лет.
Должности руководителей, заместителей руководителей государственных и муниципальных научных организаций и руководителей их филиалов замещаются лицами в возрасте не старше семидесяти лет независимо от срока действия трудовых договоров. Лица, замещающие указанные должности и достигшие возраста семидесяти лет, переводятся с их письменного согласия на иные должности, соответствующие их квалификации.
Одно и то же лицо не может замещать должность руководителя одной и той же государственной или муниципальной научной организации более трех сроков, если иное не предусмотрено федеральными законами или решениями Президента Российской Федерации.
В отдельных случаях, предусмотренных федеральными законами или решениями Президента Российской Федерации, срок пребывания руководителя государственной или муниципальной научной организации в своей должности по достижении им возраста, уста

['С руководителями государственных и муниципальных научных организаций заключаются трудовые договоры на срок до пяти лет.',
 'Должности руководителей, заместителей руководителей государственных и муниципальных научных организаций и руководителей их филиалов замещаются лицами в возрасте не старше семидесяти лет независимо от срока действия трудовых договоров.',
 'Лица, замещающие указанные должности и достигшие возраста семидесяти лет, переводятся с их письменного согласия на иные должности, соответствующие их квалификации.',
 'Одно и то же лицо не может замещать должность руководителя одной и той же государственной или муниципальной научной организации более трех сроков, если иное не предусмотрено федеральными законами или решениями Президента Российской Федерации.',
 'В отдельных случаях, предусмотренных федеральными законами или решениями Президента Российской Федерации, срок пребывания руководителя государственной или муниципальной научной организации в своей должности по достижении

In [32]:


df.insert(loc=len(df.columns), column='text2embed', value='', allow_duplicates=True)
df.text2embed = df.text.apply(lambda x: get_sentence_spans(x))

In [33]:
df.head(10)

,Unnamed: 0,party,section_number,section_text,chapter_number,chapter_text,article_number,article_text,text,href,text2embed
0,501,ЧАСТЬ ПЯТАЯ,Раздел XIII,ЗАЩИТА ТРУДОВЫХ ПРАВ И СВОБОД. РАССМОТРЕНИЕ И ...,Глава 61,РАССМОТРЕНИЕ И РАЗРЕШЕНИЕ КОЛЛЕКТИВНЫХ ТРУДОВЫ...,Статья 405,Гарантии в связи с разрешением коллективного т...,"Члены примирительной комиссии, трудовые арбитр...",http://www.consultant.ru/document/cons_doc_LAW...,"[Члены примирительной комиссии, трудовые арбит..."
1,137,ЧАСТЬ ТРЕТЬЯ,Раздел V,ВРЕМЯ ОТДЫХА,Глава 19,ОТПУСКА,Статья 126,Замена ежегодного оплачиваемого отпуска денежн...,"Часть ежегодного оплачиваемого отпуска, превыш...",http://www.consultant.ru/document/cons_doc_LAW...,"[Часть ежегодного оплачиваемого отпуска, превы..."
2,405,ЧАСТЬ ЧЕТВЕРТАЯ,Раздел XII,ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА ОТДЕЛЬНЫХ КАТЕ...,Глава 53.1,"ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА РАБОТНИКОВ, НА...",Статья 341.1,Общие положения,Частное агентство занятости или другое юридиче...,http://www.consultant.ru/document/cons_doc_LAW...,[Частное агентство занятости или другое юридич...
3,0,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 1,Цели и задачи трудового законодательства,Целями трудового законодательства являются уст...,http://www.consultant.ru/document/cons_doc_LAW...,[Целями трудового законодательства являются ус...
4,253,ЧАСТЬ ТРЕТЬЯ,Раздел X,ОХРАНА ТРУДА,Глава 36,УПРАВЛЕНИЕ ОХРАНОЙ ТРУДА (введено Федеральным ...,Статья 225,Финансирование мероприятий по улучшению услови...,Финансирование мероприятий по улучшению услови...,http://www.consultant.ru/document/cons_doc_LAW...,[Финансирование мероприятий по улучшению услов...
5,317,ЧАСТЬ ЧЕТВЕРТАЯ,Раздел XII,ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА ОТДЕЛЬНЫХ КАТЕ...,Глава 43,ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА РУКОВОДИТЕЛЯ О...,Статья 281,Особенности регулирования труда членов коллеги...,"Федеральными законами, учредительными документ...",http://www.consultant.ru/document/cons_doc_LAW...,"[Федеральными законами, учредительными докумен..."
6,448,ЧАСТЬ ПЯТАЯ,Раздел XIII,ЗАЩИТА ТРУДОВЫХ ПРАВ И СВОБОД. РАССМОТРЕНИЕ И ...,Глава 57,ФЕДЕРАЛЬНЫЙ ГОСУДАРСТВЕННЫЙ КОНТРОЛЬ (НАДЗОР) ...,Статья 353,Федеральный государственный контроль (надзор) ...,Федеральный государственный контроль (надзор) ...,http://www.consultant.ru/document/cons_doc_LAW...,[Федеральный государственный контроль (надзор)...
7,11,ЧАСТЬ ПЕРВАЯ,Раздел I,ОБЩИЕ ПОЛОЖЕНИЯ,Глава 1,ОСНОВНЫЕ НАЧАЛА ТРУДОВОГО ЗАКОНОДАТЕЛЬСТВА,Статья 13,Действие трудового законодательства и иных акт...,Федеральные законы и иные нормативные правовые...,http://www.consultant.ru/document/cons_doc_LAW...,[Федеральные законы и иные нормативные правовы...
8,461,ЧАСТЬ ПЯТАЯ,Раздел XIII,ЗАЩИТА ТРУДОВЫХ ПРАВ И СВОБОД. РАССМОТРЕНИЕ И ...,Глава 57,ФЕДЕРАЛЬНЫЙ ГОСУДАРСТВЕННЫЙ КОНТРОЛЬ (НАДЗОР) ...,Статья 365,Взаимодействие федеральной инспекции труда с г...,Федеральная инспекция труда осуществляет свою ...,http://www.consultant.ru/document/cons_doc_LAW...,[Федеральная инспекция труда осуществляет свою...
9,450,ЧАСТЬ ПЯТАЯ,Раздел XIII,ЗАЩИТА ТРУДОВЫХ ПРАВ И СВОБОД. РАССМОТРЕНИЕ И ...,Глава 57,ФЕДЕРАЛЬНЫЙ ГОСУДАРСТВЕННЫЙ КОНТРОЛЬ (НАДЗОР) ...,Статья 354,Федеральная инспекция труда,Федеральная инспекция труда - единая централиз...,http://www.consultant.ru/document/cons_doc_LAW...,[Федеральная инспекция труда - единая централи...


In [34]:
df.to_excel("fixed.xlsx")

In [15]:
!pip list | grep tensorflow

# Prepare indices

In [35]:
index_name = 'labor_law'

# source_no_vecs = ['title', 'db_id']
# vector_dims = 512
vector_dims = 768
# vector_dims = 312

source_law = 'Трудовой кодекс'

columns_to_add_as_text = [
    "party",
    "section_number",
    "section_text",
    "chapter_number",
    "chapter_text",
    "article_number",
    "article_text",
    "href",
    # "text",
]

#### Optional: clear indices

In [46]:
remove_indices = True
if remove_indices:
    es.indices.delete(index=index_name, ignore=[400, 404])
    es.indices.flush(index="*")
    # es.indices.shrink(index="*")
if es.indices.exists(index=index_name):
    es.indices.get_mapping(index=index_name)

## Settings

In [47]:
settings = {
    "number_of_shards": 1,
    "number_of_replicas": 0,
    "elastiknn": True,    
  # "settings": {
  #   "number_of_shards": 1,
  #   "number_of_replicas": 0,
  #   "elastiknn": True,    
  # }
    
    "analysis": {
      "filter": {
        "russian_stop": {
          "type":       "stop",
          "stopwords":  "_russian_"
        },
        "russian_keywords": {
          "type":       "keyword_marker",
          "keywords":   ["пример", "кодекс"]
        },
        "russian_stemmer": {
          "type":       "stemmer",
          "language":   "russian"
        }
      },
      "analyzer": {
        "rebuilt_russian": {
          "tokenizer":  "standard",
          "filter": [
            "lowercase",
            "russian_stop",
            "russian_keywords",
            "russian_stemmer"
          ]
        }
      }
    }
}

mapping = {
  "dynamic": False,
  "properties": {
    "source_law" : {"type": "keyword"},
    "original_row": {"type": "long"},
      
    "doc_structure_tags": {"type": "keyword"},
    
    "party": {"type": "keyword" },
    "section_number": {"type": "keyword" },
    "section_text": {"type": "text", "analyzer": "rebuilt_russian" },
    "chapter_number": {"type": "keyword" },
    "chapter_text": {"type": "text", "analyzer": "rebuilt_russian" },
    "article_number": {"type": "keyword" },
    "article_text": {"type": "text", "analyzer": "rebuilt_russian" },
    "href": {"type": "text" },
    "text": {"type": "text", "store": True, "analyzer": "rebuilt_russian" },
      

    "text_emb": {
        "type": "elastiknn_dense_float_vector", # 1
        "elastiknn": {
            "dims": vector_dims,                        # 2
            "model": "lsh",                     # 3
            "similarity": "cosine",             # 4
            "L": 300,                            # 5
            "k": 10                              # 6
        }
    },

  }
}

In [48]:
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, settings=settings)
    es.indices.put_mapping(mapping, index=index_name,  )

/tmp/ipykernel_539348/4126782526.py:3: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.put_mapping(mapping, index=index_name,  )


In [49]:
es.cluster.put_settings({
    "persistent": { "cluster.max_shards_per_node": "5000" },
    "transient": {
        "cluster.routing.allocation.total_shards_per_node": 5100
    }
})

/tmp/ipykernel_539348/3899948553.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.cluster.put_settings({


{'acknowledged': True,
 'persistent': {'cluster': {'max_shards_per_node': '5000'}},
 'transient': {'cluster': {'routing': {'allocation': {'total_shards_per_node': '5100'}}}}}

In [50]:
es.indices.get_settings(index=index_name)

{'labor_law': {'settings': {'index': {'elastiknn': 'true',
    'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'labor_law',
    'creation_date': '1677503325359',
    'analysis': {'filter': {'russian_stemmer': {'type': 'stemmer',
       'language': 'russian'},
      'russian_stop': {'type': 'stop', 'stopwords': '_russian_'},
      'russian_keywords': {'keywords': ['пример', 'кодекс'],
       'type': 'keyword_marker'}},
     'analyzer': {'rebuilt_russian': {'filter': ['lowercase',
        'russian_stop',
        'russian_keywords',
        'russian_stemmer'],
       'tokenizer': 'standard'}}},
    'number_of_replicas': '0',
    'uuid': 'N_5y_oU1Q0qdaDL5VmOf5A',
    'version': {'created': '7170399'}}}}}

In [52]:
!curl -X POST "localhost:9200/labor_law/_analyze?pretty" -H 'Content-Type: application/json' -d'{"field": "text", "text": "трудового кодекса"}'
!curl -X POST "localhost:9200/labor_law/_analyze?pretty" -H 'Content-Type: application/json' -d'{"field": "text","text": "трудовым кодексом"}'


{
  "tokens" : [
    {
      "token" : "трудов",
      "start_offset" : 0,
      "end_offset" : 9,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "кодекс",
      "start_offset" : 10,
      "end_offset" : 17,
      "type" : "<ALPHANUM>",
      "position" : 1
    }
  ]
}
{
  "tokens" : [
    {
      "token" : "трудов",
      "start_offset" : 0,
      "end_offset" : 8,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "кодекс",
      "start_offset" : 9,
      "end_offset" : 17,
      "type" : "<ALPHANUM>",
      "position" : 1
    }
  ]
}


In [53]:
es.indices.get_mapping(index=index_name)

{'labor_law': {'mappings': {'dynamic': 'false',
   'properties': {'article_number': {'type': 'keyword'},
    'article_text': {'type': 'text', 'analyzer': 'rebuilt_russian'},
    'chapter_number': {'type': 'keyword'},
    'chapter_text': {'type': 'text', 'analyzer': 'rebuilt_russian'},
    'doc_structure_tags': {'type': 'keyword'},
    'href': {'type': 'text'},
    'original_row': {'type': 'long'},
    'party': {'type': 'keyword'},
    'section_number': {'type': 'keyword'},
    'section_text': {'type': 'text', 'analyzer': 'rebuilt_russian'},
    'source_law': {'type': 'keyword'},
    'text': {'type': 'text', 'store': True, 'analyzer': 'rebuilt_russian'},
    'text_emb': {'type': 'elastiknn_dense_float_vector',
     'elastiknn': {'L': 300,
      'dims': 768,
      'k': 10,
      'model': 'lsh',
      'similarity': 'cosine'}}}}}}

# Generate and Index Embeddings

In [54]:

import numpy as np

from tqdm import tqdm

## prepare embedder

In [55]:
# import tensorflow_hub as hub
# import tensorflow_text
# model_name = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
# model_name = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3'
# embed = hub.load(model_name)

from transformers import AutoTokenizer, AutoModel
import torch


# model_name = 'symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli'
# model_name = 'inkoziev/sbert_pq'
model_name = 'sentence-transformers/distiluse-base-multilingual-cased-v1'

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_sentence_embeddings(sentences):
    # Tokenize sentences
    encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, mean pooling.
    return mean_pooling(model_output, encoded_input['attention_mask'])


/home/toor/semantic-search/.venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
get_sentence_embeddings('Лица, замещающие указанные должности и достигшие возраста семидесяти лет, переводятся с их письменного согласия на иные должности, соответствующие их квалификации.').shape #.numpy()[0]

torch.Size([1, 768])

In [57]:
structure_columns = ['party', 'section_number', 'chapter_number', 'article_number']
def collect_row_tags(row):
    structure_path = ''
    structure_tags = []
    for c in structure_columns:
        structure_path += row[c]    
        structure_tags.append(structure_path)
        structure_path += '/'
    return structure_tags

collect_row_tags({structure_columns[0]:'ЧАСТЬ ПЕРВАЯ',structure_columns[1]:'Раздел I',structure_columns[2]:'Глава 1',structure_columns[3]:'Статья 1'})

['ЧАСТЬ ПЕРВАЯ',
 'ЧАСТЬ ПЕРВАЯ/Раздел I',
 'ЧАСТЬ ПЕРВАЯ/Раздел I/Глава 1',
 'ЧАСТЬ ПЕРВАЯ/Раздел I/Глава 1/Статья 1']

In [58]:
tqdm.pandas(desc="pandas processing...")

def heads_actions():
    for index, row in df.iterrows():
        embs=row['text2embed']
        tags = collect_row_tags(row)
        for e in embs:
            d = {
              "_op_type": "index", 
              "_index": index_name, 

              "original_row": index, 
              "source_law": source_law, 
            }
            for c in columns_to_add_as_text:
                d[c] = row[c]

            d['doc_structure_tags'] = tags
            d['text'] = e
            # d['text_emb'] = embed(e).numpy()[0]
            d['text_emb'] = get_sentence_embeddings(e).numpy()[0]
            yield d

bulk(es, heads_actions(), chunk_size=1000, max_retries=2)

/home/toor/semantic-search/.venv/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


(2876, [])

In [59]:
es.indices.refresh(index=index_name)
es.indices.forcemerge(index=index_name, max_num_segments=1, request_timeout=120)

{'_shards': {'total': 1, 'successful': 1, 'failed': 0}}

In [45]:
def vector_actions():
    for index, row in df.iterrows():
        yield { 
            "_op_type": "update", 
            "_index": index_name, 
            
            "_id": index, 
            "doc": { 
              "text_emb": { "values": embed(row['text2embed']).numpy()[0] }
            }
        }
        

# bulk(es, vector_actions(), chunk_size=50, max_retries=10, request_timeout=60)

C:\projects\semantic-search-td\.venv\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


(521, [])

In [46]:
es.indices.refresh(index=index_name)
es.indices.forcemerge(index=index_name, max_num_segments=1, request_timeout=300)

{'_shards': {'total': 1, 'successful': 1, 'failed': 0}}

In [71]:
es.get(index=index_name, id="1")


NotFoundError: NotFoundError(404, '{"_index":"labor_law","_type":"_doc","_id":"1","found":false}')

# Search experiments

In [60]:
def display_hits(res):
    print(f"Found {res['hits']['total']['value']} hits in {res['took']} ms. Showing top {len(res['hits']['hits'])}.")
    print("")
    for hit in res['hits']['hits']:
        s = hit['_source']
        # print(f"Id:{s.get('db_id', None)} Title   {s.get('title', None)}")
        print(f"({hit.get('_id')}) {s.get('source_law')}: {s.get('party')} > {s.get('section_number')} {s.get('section_text')} > {s.get('chapter_number')} {s.get('chapter_text')} >")
        print(s.get('text'))
        print(f"Score   {hit.get('_score', None)}")


## Keyword search

In [61]:

body = {
  "query": {
    "multi_match": {
      "query": "Основные начала",
      "fields": ["text^2"]
    }
  }
}

res = es.search(index=index_name, body=body, size=5, _source=columns_to_add_as_text+['source_law', 'text'])
display_hits(res)

Found 194 hits in 39 ms. Showing top 5.

(82YCk4YB50Sp-JtRovXO) Трудовой кодекс: ЧАСТЬ ТРЕТЬЯ > Раздел V ВРЕМЯ ОТДЫХА > Глава 19 ОТПУСКА >
О времени начала отпуска работник должен быть извещен под роспись не позднее чем за две недели до его начала.
Score   14.983728
(9mYCk4YB50Sp-JtRovTN) Трудовой кодекс: ЧАСТЬ ТРЕТЬЯ > Раздел VI ОПЛАТА И НОРМИРОВАНИЕ ТРУДА > Глава 21 ЗАРАБОТНАЯ ПЛАТА >
Оплата отпуска производится не позднее чем за три дня до его начала.
Score   13.319426
(kGYEk4YB50Sp-JtRP_gb) Трудовой кодекс: ЧАСТЬ ТРЕТЬЯ > Раздел V ВРЕМЯ ОТДЫХА > Глава 19 ОТПУСКА >
Если работнику своевременно не была произведена оплата за время ежегодного оплачиваемого отпуска либо работник был предупрежден о времени начала этого отпуска позднее чем за две недели до его начала, то работодатель по письменному заявлению работника обязан перенести ежегодный оплачиваемый отпуск на другой срок, согласованный с работником.
Score   12.088051
(lmYCk4YB50Sp-JtRovPM) Трудовой кодекс: ЧАСТЬ ВТОРАЯ > Раздел II 

/tmp/ipykernel_539348/2773970549.py:10: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index=index_name, body=body, size=5, _source=columns_to_add_as_text+['source_law', 'text'])


In [40]:
res

{'took': 49,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 48, 'relation': 'eq'},
  'max_score': 14.874731,
  'hits': [{'_index': 'labor_law',
    '_type': '_doc',
    '_id': 'PWYdc4YB50Sp-JtRM9-x',
    '_score': 14.874731,
    '_source': {'section_text': 'ВРЕМЯ ОТДЫХА',
     'article_number': 'Статья 123',
     'chapter_number': 'Глава 19',
     'chapter_text': 'ОТПУСКА',
     'article_text': 'Очередность предоставления ежегодных оплачиваемых отпусков',
     'source_law': 'Трудовой кодекс',
     'href': 'http://www.consultant.ru/document/cons_doc_LAW_34683/627272a057d8634b0366744b16b04a6853d96fad/',
     'text': 'О времени начала отпуска работник должен быть извещен под роспись не позднее чем за две недели до его начала.',
     'party': 'ЧАСТЬ ТРЕТЬЯ',
     'section_number': 'Раздел V'}},
   {'_index': 'labor_law',
    '_type': '_doc',
    '_id': 'PmYdc4YB50Sp-JtRM96w',
    '_score': 12.886115,
    '_source': {'

## Terms Aggregation

In [41]:

body = {
  "aggs": {
    "parties": {
      "terms": { 
          "field": "party",
          "order": { "_key": "asc" },
      }
    },
    "sections": {
        "terms": {
            "field": "section_number", 
            "order": { "_key": "asc" },
        },
        
        "aggs": {
            "chapter_numbers": {  
                "terms": {
                    "field": "chapter_number", 
                    "order": { "_key": "asc" },
                },
            }
      }

    }
    
  }
}
body = {
  "aggs": {
    "genres": {
      "terms": { "field": "party" }
    }
  }
}
es.search(index=index_name, body=body, size=0 )

/tmp/ipykernel_298719/3900000421.py:35: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.search(index=index_name, body=body, size=0 )


{'took': 45,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2920, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'genres': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 0,
   'buckets': [{'key': 'ЧАСТЬ ТРЕТЬЯ', 'doc_count': 1364},
    {'key': 'ЧАСТЬ ЧЕТВЕРТАЯ', 'doc_count': 665},
    {'key': 'ЧАСТЬ ПЯТАЯ', 'doc_count': 370},
    {'key': 'ЧАСТЬ ПЕРВАЯ', 'doc_count': 265},
    {'key': 'ЧАСТЬ ВТОРАЯ', 'doc_count': 203},
    {'key': 'ЧАСТЬ ШЕСТАЯ', 'doc_count': 53}]}}}

## Semantic search

In [42]:
# query_vec = embed("Как устанавливается оплата на севере").numpy()[0]
query_vec = get_sentence_embeddings("Как устанавливается оплата на севере").numpy()[0]

In [43]:
body = {
    "query": {
        "elastiknn_nearest_neighbors": {
            "field": "text_emb",                     # 1
            "vec": {                               # 2
                "values": query_vec
            },
            "model": "lsh",                        # 3
            "similarity": "cosine",                # 4
            "candidates": 50                       # 5
        }
    }
}

res = es.search(index=index_name, body=body, size=10, _source=columns_to_add_as_text+['source_law','text'])
display_hits(res)

Found 50 hits in 45 ms. Showing top 10.

(q2Ydc4YB50Sp-JtRWuAF) Трудовой кодекс: ЧАСТЬ ЧЕТВЕРТАЯ > Раздел XII ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА ОТДЕЛЬНЫХ КАТЕГОРИЙ РАБОТНИКОВ > Глава 50 ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА ЛИЦ, РАБОТАЮЩИХ В РАЙОНАХ КРАЙНЕГО СЕВЕРА И ПРИРАВНЕННЫХ К НИМ МЕСТНОСТЯХ (в ред. Федерального закона от 30.06.2006 N 90-ФЗ) >
Окончательный расчет производится по возвращении из отпуска на основании предоставленных билетов или других документов.
Score   1.8844134
(smYdc4YB50Sp-JtRM9-y) Трудовой кодекс: ЧАСТЬ ЧЕТВЕРТАЯ > Раздел XII ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА ОТДЕЛЬНЫХ КАТЕГОРИЙ РАБОТНИКОВ > Глава 41 ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА ЖЕНЩИН, ЛИЦ С СЕМЕЙНЫМИ ОБЯЗАННОСТЯМИ >
Порядок предоставления указанных дополнительных оплачиваемых выходных дней устанавливается Правительством Российской Федерации.
Score   1.8759553
(72Ydc4YB50Sp-JtRM92v) Трудовой кодекс: ЧАСТЬ ПЕРВАЯ > Раздел I ОБЩИЕ ПОЛОЖЕНИЯ > Глава 2 ТРУДОВЫЕ ОТНОШЕНИЯ, СТОРОНЫ ТРУДОВЫХ ОТНОШЕНИЙ, ОСНОВАНИЯ ВОЗНИКНОВЕ

/tmp/ipykernel_298719/1155692535.py:15: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index=index_name, body=body, size=10, _source=columns_to_add_as_text+['source_law','text'])


In [87]:
body = {
    "query": {
        "elastiknn_nearest_neighbors": {
            "field": "text_emb",                     # 1
            "vec": {                               # 2
                "values": query_vec
            },
            "model": "exact",                        # 3
            "similarity": "cosine",                # 4
            "candidates": 50                       # 5
        }
    }
}

res = es.search(index=index_name, body=body, size=10, _source=columns_to_add_as_text+['text', 'source_law'])
display_hits(res)

Found 2920 hits in 94 ms. Showing top 10.

(mEeFU4YBz92FDRBYTJV2) Трудовой кодекс: ЧАСТЬ ЧЕТВЕРТАЯ > Раздел XII ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА
ОТДЕЛЬНЫХ КАТЕГОРИЙ РАБОТНИКОВ > Глава 50 ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА ЛИЦ, РАБОТАЮЩИХ
В РАЙОНАХ КРАЙНЕГО СЕВЕРА И ПРИРАВНЕННЫХ К НИМ МЕСТНОСТЯХ
(в ред. Федерального закона от 30.06.2006 N 90-ФЗ) >
Окончательный расчет производится по возвращении из отпуска на основании предоставленных билетов или других документов.
Score   1.8844135
(n0eFU4YBz92FDRBYHJRJ) Трудовой кодекс: ЧАСТЬ ЧЕТВЕРТАЯ > Раздел XII ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ ТРУДА
ОТДЕЛЬНЫХ КАТЕГОРИЙ РАБОТНИКОВ > Глава 41 ОСОБЕННОСТИ РЕГУЛИРОВАНИЯ
ТРУДА ЖЕНЩИН, ЛИЦ С СЕМЕЙНЫМИ ОБЯЗАННОСТЯМИ >
Порядок предоставления указанных дополнительных оплачиваемых выходных дней устанавливается Правительством Российской Федерации.
Score   1.8759553
(3EeFU4YBz92FDRBYHJJG) Трудовой кодекс: ЧАСТЬ ПЕРВАЯ > Раздел I ОБЩИЕ ПОЛОЖЕНИЯ > Глава 2 ТРУДОВЫЕ ОТНОШЕНИЯ, СТОРОНЫ
ТРУДОВЫХ ОТНОШЕНИЙ, ОСНОВАНИЯ ВОЗНИКНО

C:\Users\artif\AppData\Local\Temp\ipykernel_35020\3725494874.py:15: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index=index_name, body=body, size=10, _source=columns_to_add_as_text+['text', 'source_law'])


## Faceted search

In [66]:
aggs_sequence = ["party", "section_number", "chapter_number", "article_number"]

def display_aggs(hits):
    aggs = hits['aggregations']
    
    def reccurent_aggs(aggs, level = 0):
        buckets = aggs['buckets']
        for b in buckets:
            indent = '\t'*level
            print(f"{indent}{b['key']}: {b['doc_count']}")
            if level+1 < len(aggs_sequence) and aggs_sequence[level+1]+'_a' in b.keys():
                reccurent_aggs(b[aggs_sequence[level+1]+'_a'], level+1)
                
    reccurent_aggs(aggs[aggs_sequence[0]+'_a'], 0)

In [67]:
top_k = 50
min_score = 1.
def filter_body(filters):
    fs = []
    for x in filters:
        if type(x) is tuple:
            fs.append({"term":{x[0]:x[1]}})
        elif type(x) is str:
            t = x.split(':', maxsplit=1)
            fs.append({"term":{t[0]:t[1]}})
    return fs
        
body = {
    "size": top_k, "min_score": min_score, 
    "query": {
        "bool": {
            "must":{
                "elastiknn_nearest_neighbors": {
                    "field": "text_emb",                     # 1
                    "vec": {                               # 2
                        "values": query_vec
                    },
                    "model": "exact",                        # 3
                    "similarity": "cosine",                # 4
                    "candidates": 50                       # 5
                }
            },
            "filter": [
                { "terms": {"article_number": ["Статья 130"]} },
                {"term": {"party":"ЧАСТЬ ТРЕТЬЯ"}}
            ] 
                # filter_body({"chapter_number": "Глава 21"}),
                # filter_body({"section_number": "Раздел VI"}),
            # filter_body(['article_number:Статья 130', 'article_number:Статья 131']),
            # [
            #     { "term": {"chapter_number": "Глава 21"} },
            # ]
        }
    },
    
    "aggs": {
        "party_a": {
            "terms": { "field": "party" },
            "aggs": {
                "section_number_a": {
                    "terms": { "field": "section_number" },
                    "aggs": {
                        "chapter_number_a": {
                            "terms": { "field": "chapter_number" },
                            "aggs": {
                                "article_number_a": {
                                    "terms": { "field": "article_number" }
                                }        
                            }
                        },
                    },  
                },
            }

        },

    },
}

res = es.search(index=index_name, body=body, size=10, _source=columns_to_add_as_text)
# display_hits(res)
res['aggregations']

C:\Users\artif\AppData\Local\Temp\ipykernel_35020\2027920009.py:65: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.search(index=index_name, body=body, size=10, _source=columns_to_add_as_text)


{'party_a': {'doc_count_error_upper_bound': 0,
  'sum_other_doc_count': 0,
  'buckets': [{'key': 'ЧАСТЬ ТРЕТЬЯ',
    'doc_count': 8,
    'section_number_a': {'doc_count_error_upper_bound': 0,
     'sum_other_doc_count': 0,
     'buckets': [{'key': 'Раздел VI',
       'doc_count': 8,
       'chapter_number_a': {'doc_count_error_upper_bound': 0,
        'sum_other_doc_count': 0,
        'buckets': [{'key': 'Глава 20',
          'doc_count': 8,
          'article_number_a': {'doc_count_error_upper_bound': 0,
           'sum_other_doc_count': 0,
           'buckets': [{'key': 'Статья 130', 'doc_count': 8}]}}]}}]}}]}}

In [151]:
display_hits(res)

Found 10 hits in 138 ms. Showing top 10.

(nJzFM4YBCU9JJJ_aoApw) None: ЧАСТЬ ТРЕТЬЯ > Раздел VI ОПЛАТА И НОРМИРОВАНИЕ ТРУДА > Глава 20 ОБЩИЕ ПОЛОЖЕНИЯ >
None
Score   1.224844
(mJzFM4YBCU9JJJ_aoApw) None: ЧАСТЬ ТРЕТЬЯ > Раздел VI ОПЛАТА И НОРМИРОВАНИЕ ТРУДА > Глава 20 ОБЩИЕ ПОЛОЖЕНИЯ >
None
Score   1.213197
(lJzFM4YBCU9JJJ_aoApw) None: ЧАСТЬ ТРЕТЬЯ > Раздел VI ОПЛАТА И НОРМИРОВАНИЕ ТРУДА > Глава 20 ОБЩИЕ ПОЛОЖЕНИЯ >
None
Score   1.1733992
(k5zFM4YBCU9JJJ_aoApw) None: ЧАСТЬ ТРЕТЬЯ > Раздел VI ОПЛАТА И НОРМИРОВАНИЕ ТРУДА > Глава 20 ОБЩИЕ ПОЛОЖЕНИЯ >
None
Score   1.1616367
(lpzFM4YBCU9JJJ_aoApw) None: ЧАСТЬ ТРЕТЬЯ > Раздел VI ОПЛАТА И НОРМИРОВАНИЕ ТРУДА > Глава 20 ОБЩИЕ ПОЛОЖЕНИЯ >
None
Score   1.1406938
(mZzFM4YBCU9JJJ_aoApw) None: ЧАСТЬ ТРЕТЬЯ > Раздел VI ОПЛАТА И НОРМИРОВАНИЕ ТРУДА > Глава 20 ОБЩИЕ ПОЛОЖЕНИЯ >
None
Score   1.1391164
(l5zFM4YBCU9JJJ_aoApw) None: ЧАСТЬ ТРЕТЬЯ > Раздел VI ОПЛАТА И НОРМИРОВАНИЕ ТРУДА > Глава 20 ОБЩИЕ ПОЛОЖЕНИЯ >
None
Score   1.1180427
(mpzFM4YBCU9JJJ_aoApw)

In [94]:
display_aggs(res)

ЧАСТЬ ТРЕТЬЯ: 1
	Раздел VI: 1
		Глава 20: 1
			Статья 130: 1
